In [37]:
import os
import numpy as np
from cv2 import cv2
import tensorflow as tf
from utils import ignoreWarnings, useDevice, LoadModel, showImage
from dataprcs import readImage, InvertLabel
from settings import IMG_HEIGHT, IMG_WIDTH, CHANNELS, IMG_SIZE, CLASSES, numofSamp, RESIZE_HEIGHT, RESIZE_WIDTH
from UNetExtended import CreateUnetEx
import time
from tqdm.notebook import tqdm

In [19]:
def InvertLabel_1(labeled_img):
  
  img = np.zeros((len(labeled_img), len(labeled_img[0,]), 3), dtype = np.uint8)

  LB = [0, 255, 255] #Light Blue
  DB = [0, 0, 255] #Dark Blue
  G = [0, 255, 0] #Green
  W = [255, 255, 255] #White
  Y = [255, 255, 0] #Yellow
  R = [255, 0, 0] #Red

  pbar = tqdm(total = len(img), desc = 'Inverting Labels', unit = 'pixels', unit_scale = len(img[0,]))

  for x in range(len(img)):
    for y in range(len(img[0,])):
      if labeled_img[x, y] == 0:
        img[x, y, :] = LB
      elif labeled_img[x, y] == 1:
        img[x, y, :] = DB 
      elif labeled_img[x, y] == 2:
        img[x, y, :] = G 
      elif labeled_img[x, y] == 3:
        img[x, y, :] = W 
      elif labeled_img[x, y] == 4:
        img[x, y, :] = Y 
      elif labeled_img[x, y] == 5:
        img[x, y, :] = R
      # else:
      #   img[x, y, :] = W
    time.sleep(0.0001)
    pbar.update(1)
  
  pbar.close()

  return img

In [3]:
def predict(model, X_test):
    print('predicting...')
    result = model.predict(X_test, verbose = 1)
        
    result = result.reshape((RESIZE_HEIGHT, RESIZE_WIDTH, CLASSES))
    result = np.argmax(result, axis = 2)
    # result = np.round(result, 0)
#     result = result.astype('int')

    return result

In [4]:
model_name = 'U-Net'
ignoreWarnings()
useDevice('CPU')

# time_stamp = input('enter time stamp: ')
time_stamp = '23-33-59'

model = LoadModel(time_stamp, model_name, weights = True, IMG_SIZE = IMG_SIZE)

impath = 'data/RGB/top_potsdam_5_12_RGB.tif'
label_impath = 'raw_data/Label/top_potsdam_3_10_label.tif.npy'

# X_test = np.load(impath)
# y_test = np.load(label_impath)
X_test = readImage(impath, IMG_SIZE)



using CPU
device: XLA_CPU device
loading model...


In [5]:
result = model.predict(X_test, verbose = 1)

8/8 [==============================] - 158s 20s/step


In [40]:
output = result
output = output.reshape((RESIZE_HEIGHT, RESIZE_WIDTH, CLASSES))
output = np.argmax(output, axis = 2)
output = InvertLabel_1(output)

KeyboardInterrupt: 

In [22]:
showImage(output, waitKey = 10**5)

press any button to close the window
displaying image for 100.0 s


In [21]:
output = InvertLabel_1(output)